In [1]:
import pyodbc
import os
from IPython.core.display import display
from dotenv import load_dotenv
import pandas as pd



Connecting to SQL Server

In [2]:
load_dotenv('credentials.env')

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER='+os.environ.get('server')+';'
    'DATABASE='+os.environ.get('database')+';'
    'UID='+ os.environ.get('uid') +';'
    'PWD='+ os.environ.get('pwd')+';')

1. Retrieve top 10 products with the highest total revenue.

In [4]:
top_ten_df = pd.read_sql("WITH revenue_by_product AS "
                         "(SELECT p.Name AS ProductName, pm.Name AS Model, SUM(pod.LineTotal) AS Revenue "
                         "FROM Purchasing.PurchaseOrderDetail pod "
                         "INNER JOIN Production.Product p "
                         "ON pod.ProductID = p.ProductID "
                         "INNER JOIN Production.ProductModel pm "
                         "ON pm.ProductModelID = p.ProductModelID "
                         "GROUP BY p.Name, pm.Name "
                         "ORDER BY SUM(pod.LineTotal) DESC "
                         "OFFSET 0 ROWS "
                         "FETCH FIRST 10 ROWS ONLY) "
                         "SELECT ProductName, Model, Revenue "
                         "FROM revenue_by_product;", conn)

display(top_ten_df)

,ProductName,Model,Revenue
0,ML Mountain Pedal,ML Mountain Pedal,2709040.950
1,ML Road Pedal,ML Road Pedal,2390330.250
2,Front Brakes,Front Brakes,2277948.750
3,Rear Brakes,Rear Brakes,2277948.750
4,HL Mountain Tire,HL Mountain Tire,2092346.025
5,ML Road Tire,ML Road Tire,1866376.050
6,ML Mountain Tire,ML Mountain Tire,1800922.200
7,Touring Pedal,Touring Pedal,1766855.475
8,HL Mountain Pedal,HL Mountain Pedal,1766855.475
9,LL Mountain Pedal,LL Mountain Pedal,1766560.950
